In [1]:
import numpy as np
import pandas as pd  
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
import pyarrow.feather as feather
from sklearn.model_selection import GridSearchCV
# Suppress warnings 
# (sometimes you might want to ignore warnings, that's how you can achieve this)
import warnings
warnings.filterwarnings('ignore')
#! brew install graphviz
RSEED= 42

In [2]:
export_df = feather.read_feather("../data/cleaned_data.feather")

In [3]:
export_df.drop(['emoji','slang','information'],axis=1, inplace=True)

In [4]:
# Function to split the dataset 
def splitdataset(df):
    train_size = int(len(df) * 0.8)
    train, test = df[0:train_size], df[train_size:len(df)]
    X_train= train.drop(["interesting_message"], axis=1)
    X_test= test.drop(["interesting_message"], axis=1)
    y_train= train["interesting_message"]
    y_test= test["interesting_message"]
    return X_train, X_test, y_train, y_test

In [5]:
# Function to perform training with MSE. 
def train_tree(X_train, y_train): 
    # Creating the classifier object 
    reg_tree = DecisionTreeClassifier(class_weight="balanced")
    param_tree = {'max_leaf_nodes':[20,50,100,500],
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 5, 10, 50],
    'min_samples_leaf':[50,200,500,1000]
    }
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    grid_tree = GridSearchCV(reg_tree, param_grid=param_tree, cv=cv,scoring='balanced_accuracy', n_jobs=-1, verbose=10)
    grid_tree.fit(X_train, y_train)
    print('Best score:\n{:.2f}'.format(grid_tree.best_score_))
    print("Best parameters:\n{}".format(grid_tree.best_params_))
    print("Best model_tree:\n{}".format(grid_tree.best_estimator_))
    best_model_tree = grid_tree.best_estimator_
    return best_model_tree
    #max_depth=3, min_samples_leaf=5

In [6]:
# Function to make predictions 
def prediction(X_test, reg_tree): 
    y_pred = reg_tree.predict(X_test)
    return y_pred 

In [7]:
# Function to calculate accuracy 
def class_metrics(y_test, y_pred):
     
    accuracy = balanced_accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).round()
    print("Predicted values:\n", y_pred) 
    print("Confusion Matrix: \n", cm) 
    print("Balanced Accuracy: %.4f%%" % (accuracy * 100.0))
    print("Report : \n", classification_report(y_test, y_pred))
    return cm, accuracy

In [8]:
# Driver code 
def main(): 
    # Building Phase 
    X_train, X_test, y_train, y_test = splitdataset(export_df) 
    basemodel = train_tree(X_train, y_train)
    print(f'Decision tree has {basemodel.tree_.node_count} nodes with maximum depth {basemodel.tree_.max_depth}.')
    # Operational Phase 
    print("-----"*15)
    print("Results:\n")
    # Prediction
    y_pred = prediction(X_test, basemodel) 
    cm,accuracy= class_metrics(y_test, y_pred)
    return basemodel,y_pred,cm,accuracy

In [9]:
export_df.columns

Index(['lenght', 'emoji_size', 'slang_char', 'slang_verb', 'slang_pron',
       'slang_adp', 'slang_noun', 'slang_num', 'slang_punt', 'slang_det',
       'info_char', 'info_words', 'info_verb', 'info_pron', 'info_adp',
       'info_noun', 'info_num', 'info_punt', 'info_det', 'Sendero Segura',
       'Red de Transporte de Pasajeros', 'Mexibús', 'Metro', 'Metrobús',
       'Mexicable', 'Trolebús', 'Cablebús', 'Tren Suburbano', 'Tren Ligero',
       'Camión, Microbús, Combi', 'interesting_message', 'cause_BrokenVehicle',
       'cause_COVID19', 'cause_Counterflow', 'cause_CycleRide',
       'cause_Demonstration', 'cause_EmergencyServices', 'cause_Event',
       'cause_Explosion', 'cause_FallenTree', 'cause_Fire', 'cause_Flood',
       'cause_GasLeak', 'cause_HeavyTraffic', 'cause_Incident',
       'cause_Landslide', 'cause_Leak', 'cause_Maintenance', 'cause_March',
       'cause_Overturn', 'cause_Pilgrimage', 'cause_ProtestCamp', 'cause_Rain',
       'cause_Reopening', 'cause_Sinkhole', '

In [10]:
basemodel,y_pred,cm,accuracy=main()

Fitting 15 folds for each of 128 candidates, totalling 1920 fits
[CV 4/15; 1/128] START criterion=gini, max_depth=2, max_leaf_nodes=20, min_samples_leaf=50
[CV 1/15; 1/128] START criterion=gini, max_depth=2, max_leaf_nodes=20, min_samples_leaf=50
[CV 8/15; 1/128] START criterion=gini, max_depth=2, max_leaf_nodes=20, min_samples_leaf=50
[CV 2/15; 1/128] START criterion=gini, max_depth=2, max_leaf_nodes=20, min_samples_leaf=50
[CV 7/15; 1/128] START criterion=gini, max_depth=2, max_leaf_nodes=20, min_samples_leaf=50
[CV 3/15; 1/128] START criterion=gini, max_depth=2, max_leaf_nodes=20, min_samples_leaf=50
[CV 5/15; 1/128] START criterion=gini, max_depth=2, max_leaf_nodes=20, min_samples_leaf=50
[CV 6/15; 1/128] START criterion=gini, max_depth=2, max_leaf_nodes=20, min_samples_leaf=50
[CV 2/15; 1/128] END criterion=gini, max_depth=2, max_leaf_nodes=20, min_samples_leaf=50;, score=0.669 total time=   6.4s
[CV 7/15; 1/128] END criterion=gini, max_depth=2, max_leaf_nodes=20, min_samples_leaf

In [12]:
export_df.columns

Index(['lenght', 'emoji_size', 'slang_char', 'slang_verb', 'slang_pron',
       'slang_adp', 'slang_noun', 'slang_num', 'slang_punt', 'slang_det',
       'info_char', 'info_words', 'info_verb', 'info_pron', 'info_adp',
       'info_noun', 'info_num', 'info_punt', 'info_det', 'Sendero Segura',
       'Red de Transporte de Pasajeros', 'Mexibús', 'Metro', 'Metrobús',
       'Mexicable', 'Trolebús', 'Cablebús', 'Tren Suburbano', 'Tren Ligero',
       'Camión, Microbús, Combi', 'interesting_message', 'cause_BrokenVehicle',
       'cause_COVID19', 'cause_Counterflow', 'cause_CycleRide',
       'cause_Demonstration', 'cause_EmergencyServices', 'cause_Event',
       'cause_Explosion', 'cause_FallenTree', 'cause_Fire', 'cause_Flood',
       'cause_GasLeak', 'cause_HeavyTraffic', 'cause_Incident',
       'cause_Landslide', 'cause_Leak', 'cause_Maintenance', 'cause_March',
       'cause_Overturn', 'cause_Pilgrimage', 'cause_ProtestCamp', 'cause_Rain',
       'cause_Reopening', 'cause_Sinkhole', '

In [14]:
from sklearn.tree import export_graphviz  
features=['lenght', 'emoji_size', 'slang_char', 'slang_verb', 'slang_pron',
       'slang_adp', 'slang_noun', 'slang_num', 'slang_punt', 'slang_det',
       'info_char', 'info_words', 'info_verb', 'info_pron', 'info_adp',
       'info_noun', 'info_num', 'info_punt', 'info_det', 'Sendero Segura',
       'Red de Transporte de Pasajeros', 'Mexibús', 'Metro', 'Metrobús',
       'Mexicable', 'Trolebús', 'Cablebús', 'Tren Suburbano', 'Tren Ligero',
       'Camión, Microbús, Combi', 'cause_BrokenVehicle',
       'cause_COVID19', 'cause_Counterflow', 'cause_CycleRide',
       'cause_Demonstration', 'cause_EmergencyServices', 'cause_Event',
       'cause_Explosion', 'cause_FallenTree', 'cause_Fire', 'cause_Flood',
       'cause_GasLeak', 'cause_HeavyTraffic', 'cause_Incident',
       'cause_Landslide', 'cause_Leak', 'cause_Maintenance', 'cause_March',
       'cause_Overturn', 'cause_Pilgrimage', 'cause_ProtestCamp', 'cause_Rain',
       'cause_Reopening', 'cause_Sinkhole', 'cause_StreetWorks',
       'cause_VehicularAccident', 'cause_Waterlogging',
       'effect_CirculationRestored', 'effect_CirculationShutdown',
       'effect_Delays', 'effect_Evacuation', 'effect_FullCapacity',
       'effect_HighWaitingTime', 'effect_InterimService',
       'effect_LaneReduction', 'effect_RouteDetour', 'effect_SecuritySpeed',
       'effect_SuspensionOfService', 'effect_TrafficImpact']
# export the decision tree to a tree.dot file 
#for visualizing the plot easily anywhere 
export_graphviz(basemodel, out_file ='tree_class.dot', 
               feature_names =features) 

In [15]:
! dot -Tpng tree_class.dot > tree_class.png

In [ ]:
f = open('ml-log.txt', 'a')
f.write('Base model: Decision Tree\n Predicted values:\n {}\n Confusion Matrix:\n {}\n Balanced Accuracy:\n {} \n Model: {}'.format(y_pred,cm,accuracy,basemodel))
f.close()